In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
from pathlib import Path
import csv
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema

## 7.A

In [20]:
from pathlib import Path
import gzip
import hashlib
import shutil
import pygeohash

endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        
    return records
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    
    return flat_record
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])
df = create_flattened_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [21]:
df.head()

airline_airline_id airline_name           airline_alias airline_iata  \
0                 410   Aerocondor  ANA All Nippon Airways           2B   
1                 410   Aerocondor  ANA All Nippon Airways           2B   
2                 410   Aerocondor  ANA All Nippon Airways           2B   
3                 410   Aerocondor  ANA All Nippon Airways           2B   
4                 410   Aerocondor  ANA All Nippon Airways           2B   

  airline_icao airline_callsign airline_country  airline_active  \
0          ARD       AEROCONDOR        Portugal            True   
1          ARD       AEROCONDOR        Portugal            True   
2          ARD       AEROCONDOR        Portugal            True   
3          ARD       AEROCONDOR        Portugal            True   
4          ARD       AEROCONDOR        Portugal            True   

   src_airport_airport_id               src_airport_name  ...  \
0                  2965.0    Sochi International Airport  ...   
1                  2966.0              Astrakhan Airport  ...   
2                  2966.0              Astrakhan Airport  ...   
3                  2968.0  Chelyabinsk Balandino Airport  ...   
4                  2968.0  Chelyabinsk Balandino Airport  ...   

  dst_airport_longitude dst_airport_altitude dst_airport_timezone  \
0             49.278702                411.0                  3.0   
1             49.278702                411.0                  3.0   
2             43.081902               1054.0                  3.0   
3             49.278702                411.0                  3.0   
4             82.650703                365.0                  7.0   

  dst_airport_dst  dst_airport_tz_id  dst_airport_type  dst_airport_source  \
0               N      Europe/Moscow           airport         OurAirports   
1               N      Europe/Moscow           airport         OurAirports   
2               N      Europe/Moscow           airport         OurAirports   
3               N      Europe/Moscow           airport         OurAirports   
4               N   Asia/Krasnoyarsk           airport         OurAirports   

   codeshare equipment       key  
0      False     [CR2]  AERKZN2B  
1      False     [CR2]  ASFKZN2B  
2      False     [CR2]  ASFMRV2B  
3      False     [CR2]  CEKKZN2B  
4      False     [CR2]  CEKOVB2B  

[5 rows x 39 columns]

In [22]:
partitions = (
        ('A'), ('B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M'),
        ('N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U'), ('V'), ('W', 'X'), ('Y', 'Z')
    )

In [23]:
kv_key = []
keys = df["key"]

#had to deal with empty strings and nan issue from code provided
for key in keys:
    check = False
    first = key[0]
    if first:
        for x in partitions:
            if first in x:
                check = True
                kv_key.append("-".join(x))
    else:
        check = True
        kv_key.append("nan")
    if check != True:
        kv_key.append("nan")
        
        
df["kv_key"] = kv_key

In [24]:
df.head()

airline_airline_id airline_name           airline_alias airline_iata  \
0                 410   Aerocondor  ANA All Nippon Airways           2B   
1                 410   Aerocondor  ANA All Nippon Airways           2B   
2                 410   Aerocondor  ANA All Nippon Airways           2B   
3                 410   Aerocondor  ANA All Nippon Airways           2B   
4                 410   Aerocondor  ANA All Nippon Airways           2B   

  airline_icao airline_callsign airline_country  airline_active  \
0          ARD       AEROCONDOR        Portugal            True   
1          ARD       AEROCONDOR        Portugal            True   
2          ARD       AEROCONDOR        Portugal            True   
3          ARD       AEROCONDOR        Portugal            True   
4          ARD       AEROCONDOR        Portugal            True   

   src_airport_airport_id               src_airport_name  ...  \
0                  2965.0    Sochi International Airport  ...   
1                  2966.0              Astrakhan Airport  ...   
2                  2966.0              Astrakhan Airport  ...   
3                  2968.0  Chelyabinsk Balandino Airport  ...   
4                  2968.0  Chelyabinsk Balandino Airport  ...   

  dst_airport_altitude dst_airport_timezone dst_airport_dst dst_airport_tz_id  \
0                411.0                  3.0               N     Europe/Moscow   
1                411.0                  3.0               N     Europe/Moscow   
2               1054.0                  3.0               N     Europe/Moscow   
3                411.0                  3.0               N     Europe/Moscow   
4                365.0                  7.0               N  Asia/Krasnoyarsk   

   dst_airport_type  dst_airport_source  codeshare  equipment       key kv_key  
0           airport         OurAirports      False      [CR2]  AERKZN2B      A  
1           airport         OurAirports      False      [CR2]  ASFKZN2B      A  
2           airport         OurAirports      False      [CR2]  ASFMRV2B      A  
3           airport         OurAirports      False      [CR2]  CEKKZN2B    C-D  
4           airport         OurAirports      False      [CR2]  CEKOVB2B    C-D  

[5 rows x 40 columns]

In [25]:
df.to_parquet(path="results/kv", partition_cols=["kv_key"])

## 7.B

In [26]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [27]:
hk = []
for key in keys:
    k = str(hash_key(key))
    hk.append(k[0])
    
df["hash_key"] = hk

In [28]:
df.to_parquet(path="results/hash", partition_cols=["hash_key"])

## 7.C

In [32]:
df['src_airport_geohash'] = df.apply(
    lambda row: pygeohash.encode(row.src_airport_latitude, row.src_airport_longitude), axis=1
)
def determine_location(src_airport_geohash):
    locations = dict(
        central=pygeohash.encode(41.1544433, -96.0422378),
        west=pygeohash.encode(45.5945645, -121.1786823),
        east=pygeohash.encode(39.08344, -77.6497145)
    )
    
    distances =[] #TODO: a list of centers and distances using the pygeohash.geohash_haversine_distance function
    
    for loc in locations:
        distances.append([pygeohash.geohash_haversine_distance(locations[loc],src_airport_geohash), loc])
    
    distances.sort()
    return distances[0][1]
df['location'] = df['src_airport_geohash'].apply(determine_location)
df.to_parquet('results/geo', partition_cols=['location'])

In [33]:
df['location'].value_counts()

east       39761
west       23268
central     4634
Name: location, dtype: int64

## 7.D

In [63]:
def balance_partitions(keys, num_partitions):
    avg = len(keys)/float(num_partitions)
    partitions = []
    l = 0.0 #place holder for index value
    #make keys all strings for sort
    keys = [str(k) for k in keys]
    while l < len(keys):
        part = (keys[int(l):int(l + avg)])
        partitions.append(sorted(part))
        l +=avg
    return partitions

In [64]:
k = [23, 'One', 2, 1, "alpha", "B"]
print(balance_partitions(k,2))
print(balance_partitions(k,3))

[['2', '23', 'One'], ['1', 'B', 'alpha']]
[['23', 'One'], ['1', '2'], ['B', 'alpha']]
